<a href="https://colab.research.google.com/github/taniokah/Sports_Analysis/blob/master/SportsAnalysis%E3%82%B3%E3%83%BC%E3%83%89%E3%81%A0%E3%81%91%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# お詫び
諸般の事情でコードだけの公開と相成りました．コメントは出来る限り入れておりますが，不明な部分も多いかと存じます．よろしくお願い致します．急遽公開ということで，突貫工事で作業したので，つながりが変な部分もあるかもしれません…

# コード

## Colaboratory内での動画表示
参照元：https://colab.research.google.com/github/tugstugi/dl-colab-notebooks/blob/master/notebooks/OpenPose.ipynb

In [0]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))


動画を表示する

In [0]:
show_local_mp4_video('hoge.mp4')

## 局所特徴量を用いたパノラマ画像への各フレームの画像のマッチング

In [0]:
#マッチングさせる動画fen.mp4
#パノラマ画像back.png
#area.mp4で書き出し

#ライブラリのロード
import cv2
import numpy as np
import os

# 動画作成
fourcc = cv2.VideoWriter_fourcc('m','p','4', 'v')
video  = cv2.VideoWriter('area.mp4', fourcc, 30.0, (1750, 747))

#上位何%の特徴点を信頼するか
match_rate = 0.5

#フレーム番号確認用カウント
count = 0

# 読み込み動画
cap = cv2.VideoCapture('fen.mp4')

# マッチング&動画書き込み
while True:
  #動画をフレームごとに読み込み
  ret, image = cap.read()
  if ret == True:
    count += 1
    # 50フレームごとにフレーム番号を表示
    if count%50==0:
      print(count)
    # パノラマ画像
    img1 = cv2.imread('./back.png',0)
    img2 = image
    # 特徴点抽出にAKAZE法を使用
    akz = cv2.AKAZE_create()
    # AKAZE法によりimg1, img2の画像特徴点を計算する
    kp1, des1 = akz.detectAndCompute(img1,None)
    kp2, des2 = akz.detectAndCompute(img2,None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    # 2つの画像の特徴点をマッチさせる
    matches = bf.match(des1, des2)
    # 上位match_rate％の特徴量を確保
    matches = sorted(matches, key=lambda x: x.distance)
    m_points = matches[:int(len(matches) * match_rate)]
    # 特徴量をもとに画像を変換
    src_pts = np.float32([kp1[m.queryIdx].pt for m in m_points]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in m_points]).reshape(-1, 1, 2)
    # 透視変換行列の計算
    h, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC)
    # フレーム画像を変換
    height, width = img1.shape[0:2]
    dst_img = cv2.warpPerspective(img2, h, (width, height))
    # 書き出し
    video.write(dst_img)
  else:
    break

cap.release()
video.release()

### 結果
AKAZE特徴量による各フレームのパノラマ画像への割り振りによって，カメラのパン状態が推定できている

（特徴点の少なくなる右側だと，推定精度が若干落ちるが，ご愛嬌ということで…）


●動画の変換

In [0]:
!ffmpeg -y -v error -i area.mp4 area_c.mp4

●動画の表示

In [0]:
show_local_mp4_video('area_c.mp4')

## 対象の姿勢推定（ここは多分大丈夫なはず）
映像からのスポーツデータ解析において，選手の姿勢を推定し，評価することは一つの興味の対象です．ここでは映像からの姿勢推定アルゴリズムの一つであるPosenetを利用して，選手の姿勢推定を行う方法について紹介します．

### 映像からの姿勢推定技術
映像からの姿勢推定技術に関する研究は，長い間行われてきましたが，近年急激に躍進した印象があります．代表的なアルゴリズム（ソフトウェア）としては，OpenPose，VNect，DensePose, PoseNetなどがあるが，OpenPoseはライセンスの問題で扱いづらく，VNectは学習モデルの提供に交渉が必要，またDensPoseは3Dのメッシュ情報を推定するのが目的となっているので，今回はPoseNetを使用する．

### PoseNet
PoseNetは，Googleが提供している姿勢推定ソフトウェアで，画像から対象の姿勢を推定することが可能（あくまで画像上の関節構造であって，3次元の姿勢情報は提供されないことに注意）．簡単なお試しであれば，

https://storage.googleapis.com/tfjs-models/demos/posenet/camera.html

から体験可能．今回は，これをPythonから手軽に扱えるライブラリposenet-pytorch

https://github.com/rwightman/posenet-pytorch

を用いて，姿勢推定を行う．



posenet-pytorchは画像単位の姿勢推定なので，まず，動画データをフレーム毎の画像に変換して，imgディレクトリに格納する．

In [0]:
#area_c.mp4を用いてposenetに適用
!mkdir img
!ffmpeg -v error -i area_c.mp4 -r 30 -f image2 ./img/%08d.png

posenet-pytorchディレクトリに移動する．

In [0]:
!wget http://lab.0093.tv/sports/posenet-pytorch.zip
!unzip posenet-pytorch.zip

In [0]:
%cd posenet-pytorch

/content/posenet-pytorch


### PoseNetによる姿勢推定
もともとのposenet-pytorchライブラリのimage_demo.pyを元に改良．
パラメータの調整をしやすくするため，デフォルトにはないオプションを幾つか追加している．また，パラメータ調整以外にも，各関節座標の情報をjsonファイルとしてjsondirに格納するようにしている（デフォルトでは座標をコンソールに出力するのみ．notxtオプションをつけた場合は，出力しない）．

param1, param2の値で結果が結構変わってくるので，実用時は要チューニング．

* param1: 姿勢の推定確率の閾値
* param2: 部位の推定確率の閾値
* maxp: 推定する最大人数
* model: mobilenet_v1を使用している．デフォルトではバージョン？101
* image_dir: 入力する画像を格納しているディレクトリ
* output_dir: スケルトンモデルを上書きした画像を出力するディレクトリ
* notxt: このオプションが指定されたときは，座標情報を出力しない

Posenetでは，計17箇所の関節の推定結果を返す．

![Posenetの関節座標](https://cdn-images-1.medium.com/max/800/1*7qDyLpIT-3s4ylULsrnz8A.png)
（参照元：
https://medium.com/tensorflow/real-time-human-pose-estimation-in-the-browser-with-tensorflow-js-7dd0bc881cd5
）

In [0]:
!python image.py --param1 0.05 --param2 0.1 --maxp=5 --model 101 --image_dir ../img --output_dir ./out

Average FPS: 8.070821877852536


### 結果
Posenetによる姿勢推定の結果，ところどころ当てはまりが悪いところがあるが，関節の推定はおおよそうまく行っていることがわかる．

ただし，ここで推定された関節の座標は，2次元上で推定された座標であって，3次元の空間座標ではないことに注意する．

2次元の関節座標から，3次元への座標を推定する方法としては，

https://github.com/una-dinosauria/3d-pose-baseline

があるが，セットアップが大変なのと，計算時間がかかるので，ここでは取り扱わない．

●動画の変換

In [0]:
!ffmpeg -v error -i ./out/%08d.png -r 30 out.mp4

●動画の表示

In [0]:
show_local_mp4_video('out.mp4')

## ピスト（コート）の推定
ピスト（コート）の推定は，各フレームでの対象の位置座標を推定するのに重要になる．フェンシングのピストは以下のような構造をしており，ピストは2本の平行な直線が各領域で分割された形であることが見て取れる．

![代替テキスト](http://www.fencingfan.net/communications/images/piste_1_b.jpg)
（参照元：
http://www.fencingfan.net/communications/
）

ここではHough変換による直線検出を行うことによってピストの推定を行う．


###Hough変換
Hough変換は，古典的ではあるが有効な直線や円の検出方法として今でも用いられている．身近な利用例では（実際に使われているかどうかは分からないが…）運転時の車線検出なども，この手法を用いることで可能である．

一般的には一度エッジ検出を行い，画像内の物体の輪郭を抽出してから，Hough変換を行うことによって，直線を検出することが多い．

実際には映像中の直線要素は数多く存在することから，今回は対象となるコート（画面下側）のみを対象とし，直線抽出を行った．

posenet-pytorchのディレクトリから移動する

In [0]:
%cd ..

/content


映像中からの直線の検出
（コートの色情報を抽出し，Hough変換により直線を抽出する）

In [0]:
#2点(x1,y1)(x2,y2)から任意の座標xが与えられたときのy座標を返す
def liney(x,x1,x2,y1,y2):
  y = ((y2-y1)/(x2-x1))*(x-x1) + y1
  return(y)

#必要ライブラリのロード
import cv2
import numpy as np
import os

# 読み込み動画
cap = cv2.VideoCapture('area_c.mp4')

#対象とするコートの色情報
bgrLower = np.array([40, 100, 170])    # 抽出する色の下限(BGR)
bgrUpper = np.array([110, 180, 250])    # 抽出する色の上限(BGR)

# 抽出された色部分から，直線推定
# resの中にx=1およびx=1750でのy座標を格納する．
res = list()
while True:
  #動画をフレームごとに読み込み
  ret, image = cap.read()
  if ret == True:
    image2 = image.copy()
    #対象とするコートに関係のない部分（上半分）を黒で塗りつぶし
    cv2.rectangle(image, (1, 1), (1750, 420), (0, 0, 0), thickness=-1)
    #対象とする色部分を画像から抽出
    mask = cv2.inRange(image, bgrLower, bgrUpper)
    img2 = cv2.bitwise_and(image, image, mask=mask)
    #グレースケールに変換
    hsv = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    #エッジ検出
    edges = cv2.Canny(hsv, 200, 250)
    #Hough変換による直線検出
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 90, minLineLength = 350, maxLineGap=100)
    #推定された直線のx=1とx=1750でのy座標を格納
    tmplist = list()
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if (y1 > 420) and (y2 > 420):
              tmplist.append(int(liney(1,x1,x2,y1,y2)))
              tmplist.append(int(liney(1750,x1,x2,y1,y2)))
    res.append(tmplist)
  else:
    break

cap.release()


Hough変換による直線検出は一本の直線を複数の直線と検出してしまうことがあるので，ピストの中心を境に上側と下側の推定されたy座標の中央値を取ることによって，一つの点とする．

In [0]:
# 直線の統一化
def calpoints(p):
  # 得られたx=1とx=1750でのy座標の値を行列化する．
  # 1列目をx=1のときのy座標，2列目をx=1750のときのy座標として行列を構築する
  tmp = np.array(p)
  ld = int(len(tmp)/2)
  tmp = tmp.reshape((ld, 2))
  # 1列目(x=1)の最大値と最小値およびその中間点を計算する．
  pmi = np.min(tmp[::,0])
  pma = np.max(tmp[::,0])
  b = (pmi+pma)/2
  # 中間点より上側および下側の点のy座標の中央値をそれぞれlu, llとする．
  lu = np.round(np.median(tmp[tmp[::,0] > b,0]))
  ll = np.round(np.median(tmp[tmp[::,0] < b,0]))
  # 2列目(x=1750)の最大値と最小値およびその中間点を計算する．
  pmi = np.min(tmp[::,1])
  pma = np.max(tmp[::,1])
  b = (pmi+pma)/2
  # 中間点より上側および下側の点のy座標の中央値をそれぞれru, rlとする．
  ru = np.round(np.median(tmp[tmp[::,1] > b,1]))
  rl = np.round(np.median(tmp[tmp[::,1] < b,1]))
  # ↑の結果をresに格納し，返り値とする．
  res = [lu,ru,ll,rl]
  return(res)

# それぞれのフレームごとに↑の計算を行い，ppに格納する．
pp = list()
for i in range(len(res)):
  pp.append(calpoints(res[i]))


↑で計算された座標を元に，映像に直線を描く

In [0]:
# ライブラリのロード
import cv2
import numpy as np
import os

# 動画作成
fourcc = cv2.VideoWriter_fourcc('m','p','4', 'v')
video  = cv2.VideoWriter('line.mp4', fourcc, 30.0, (1750, 747))

# 読み込み動画
cap = cv2.VideoCapture('area_c.mp4')
while True:
  # 動画をフレームごとに読み込み
  ret, image = cap.read()
  if ret == True:
    #上側および下側の直線の描写
    cv2.line(image, (1, int(pp[count][0])), (1750, int(pp[count][1])), (0, 0, 255), 5)
    cv2.line(image, (1, int(pp[count][2])), (1750, int(pp[count][3])), (0, 0, 255), 5)
    video.write(image)
  else:
    break

cap.release()
video.release()

### 結果
Hough変換を用いることにより，コートの直線の検出を行った．今回はコートの色や領域などの事前情報を用いることでHough変換で検出しやすくしている．近年では深層学習を用いた直線検出なども提案されてきており，Hough変換はお手軽に計算できるので，利点は多いものの，今後はそれらの方法が一般化する可能性がある．

●動画の変換

In [0]:
!ffmpeg -y -v error -i line.mp4 line2.mp4

●動画の表示

In [0]:
show_local_mp4_video('line2.mp4')

## 透視投影変換行列
選手の座標を平面のコート上に表示するために，透視投影変換行列の計算を行う．今回パンを含むカメラの映像から，左側の開始線と，右側のエンドラインの内側が画面内に収まっており，この情報を使用することによって，透視投影変換行列の計算を行う．

具体的には左側の開始線と，右側のエンドラインの内側は，2m x 7mのエリアとなるので，パノラマ画像を用いて，その変換を行う．
（※事前にパノラマ画像における開始線の上側，下側，エンドラインの上側，下側の座標は手動で取得しておく）

画像表示のためのライブラリをロードする

In [0]:
from google.colab.patches import cv2_imshow

透視投影変換行列の計算

In [0]:
# ライブラリのロード
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 背景のパノラマ画像の読み込み
img = cv2.imread('./back.png') 
# 変換前の4隅の座標を入力
src_pts=np.float32([[245, 557], [1572, 506], [237, 665], [1679, 602]])
# 変換後の4隅の座標を入力
dst_pts=np.float32([[0, 0], [700, 0], [0, 200], [700, 200]])
# 変換前変換後の座標を元に，透視投影変換行列を計算
M = cv2.getPerspectiveTransform(src_pts, dst_pts)
# 実際にパノラマ画像内の領域を変換
dst = cv2.warpPerspective(img, M, (700,200))

plt.figure(figsize=(20, 20), dpi=50)
plt.subplot(121),plt.imshow(img),plt.title('Input')
plt.subplot(122),plt.imshow(dst),plt.title('Output')
plt.show()

print("透視投影変換行列")
print(M)

### ピスト内の選手の判定
透視投影変換行列が得られることにより，映像中の選手の座標も2m x 7mの平面に投影することができる．しかし，映像中には複数の人物が写り込んでいるので，対象とする選手であることを特定するため，Posenetによって推定された足の座標（今回は右足のみとした）がコートを示すに直線の内側にあるか否かで選手の判定を行う．

In [0]:
# (x1, y1), (x2, y2)の2点を通る直線と，右足の座標の位置関係の計算
# ansの値の正負で直線のどちら側にあるか判定することができる．
def lined(x1,y1,x2,y2,x,y):
  vx1 = x2 - x1
  vy1 = y2 - y1
  vx2 = x - x1
  vy2 = y - y1
  ans = vx1 * vy2 - vy1 * vx2
  return(ans)


# 各フレームにおいて，ピスト内にある選手のIDを取得する
# IDを格納するリストの作成
# coorにはPosenetで推定された関節座標をまとめたものが入っている．
ids = list()
# 各フレームで計算を行う．
for i in range(len(coor)):
  # 一時的な空リストの作成
  tmp = list()
  # 各フレームで認識されている被験者分判定を行う．
  for j in range(len(coor[i])):
    # ピストを囲む上下直線の左端(x=1)と右端(x=1750)の座標を取得
    pt = np.array([[1,pp[i][0]], [1750,pp[i][1]], [1,pp[i][2]], [1750,pp[i][3]]])
    # 被験者番号Jの右足の座標を取得
    co = coor[i][j][10][0:2]
    # 右足の座標が上下直線のどちら側にあるかを判定
    dd1 = lined(pt[0][0],pt[0][1],pt[1][0],pt[1][1],co[0],co[1])
    dd2 = lined(pt[2][0],pt[2][1],pt[3][0],pt[3][1],co[0],co[1])
    # 符号の判定
    tf1 = dd1<0
    tf2 = dd2<0
    # 符号が一致していない場合，ピスト内にいると判定する
    if (tf1==tf2)==False:
      tmp.append(j)
  ids.append(tmp)



### 疑似コートへの描写
ピスト内の選手の判定結果を元に，ピスト上での選手の座標の描写を行う．今回は右足と左足の座標および，それを直線で結んだものを描写することにする．各足の座標の変換は，先程計算した透視投影変換行列（プログラム中ではM）を使用することによって，得ることができる．

In [0]:
# ライブラリのロード
import cv2
import numpy as np

# 動画作成（コートの座標はマージンを取って3m x 9mとした）
fourcc = cv2.VideoWriter_fourcc('m','p','4', 'v')
video  = cv2.VideoWriter('result.mp4', fourcc, 30.0, (900, 300))

# 各フレーム分変換を行う
for i in range(len(coor)):
  # 空のコートの画像（3m x 9m…300px x 900px）の画像を作成する
  img = np.full((300, 900, 3), 200, dtype=np.uint8)
  # 上側の直線を引く
  cv2.line(img, (1, 50), (900, 50), (255, 255, 255), thickness=2, lineType=cv2.LINE_4)
  # 下側の直線を引く
  cv2.line(img, (1, 250), (900, 250), (255, 255, 255), thickness=2, lineType=cv2.LINE_4)
  # 右側開始線を描く
  cv2.rectangle(img, (440, 50), (460, 250), (255, 255, 255), thickness=-1)
  # 左側開始線を描く
  cv2.rectangle(img, (40, 50), (60, 250), (255, 255, 255), thickness=-1)
  # 中心線を描く
  cv2.line(img, (250, 50), (250, 250), (255, 255, 255), thickness=5, lineType=cv2.LINE_4)
  # エンドラインを描く
  cv2.rectangle(img, (750, 50), (900, 250), (255, 255, 255), thickness=-1)
  # 足の座標を描く
  for j in range(len(ids[i])):
    # 該当被験者のIDを取得
    id = ids[i][j]
    # 右足及び左足の座標を取得
    co1 = np.insert(coor[i][id][10][0:2],2,1)
    co2 = np.insert(coor[i][id][13][0:2],2,1)
    # 行列形式に変換
    co1.shape = (3,1)
    co2.shape = (3,1)
    # 透視投影変換行列によって平面上に投影
    co1 = np.dot(M,co1)
    co1 = (co1/co1[2]).T[0][0:2]+100
    co2 = np.dot(M,co2)
    co2 = (co2/co2[2]).T[0][0:2]+100
    co1 = tuple(co1.astype(np.int32))
    co2 = tuple(co2.astype(np.int32))
    # 右足と左足を線で結ぶ
    cv2.line(img, co1, co2, (0, 0, 0), thickness=3, lineType=cv2.LINE_4)
    # 右足及び左足の座標をプロットする
    cv2.circle(img, co1, 10, (255,0,0), thickness=-1)
    cv2.circle(img, co2, 10, (255,0,0), thickness=-1)
  video.write(img)
  
video.release()

### 結果
透視投影変換行列によってコートの座標に変換した結果，各選手の右足と左足の座標の試合中の変化を確認することができた．特に～10sまでは右側の選手が審判の影に隠れていることによって，表示ができていないが，その他の領域については，ノイズが交じるものの，およそプロットすることができている．

今回は投影された座標をそのまま表示しているが，実際の解析の際は，座標の変化を平滑化することによって，ノイズの変化を低減することができ，また一部推定できなかった領域については，パーティクルフィルタなどを用いることによって，保管する手法などが考えられる．

●動画の変換

In [0]:
!ffmpeg -y -v error -i result.mp4 result2.mp4

●動画の表示

In [0]:
show_local_mp4_video('result2.mp4')